In [5]:
import datetime
import apache_beam as beam
import os

No handlers could be found for logger "oauth2client.contrib.multistore_file"
/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 16:06:59.140182. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [6]:
RUNNER = "Dataflow"
PROJECT = 'ksalama-gcp-playground'
BUCKET = 'ksalama-gcs-demo'
REGION = 'europe-west1'
TOPIC = 'flights'

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 16:06:59.889724. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [9]:
def to_bq_row(message):

    data_row = message.data
    return data_row
  

#print(exract_fields("101,2.0334,-856.06"))
  
def run_flights_pipeline():
    
    job_name = 'ingest-flight-events-{}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
    print 'Launching Dataflow job {}'.format(job_name)
    print 'Check the Dataflow jobs on Google Cloud Console...'

    TMP_DIR = 'gs://{}/data/flights'.format(BUCKET)

    options = {
        'staging_location': os.path.join(TMP_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(TMP_DIR, 'tmp'),
        'job_name': job_name,
        'project': PROJECT,
        'streaming': True,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True
      }


    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    source = beam.io.PubSubSource("projects/ksalama-gcp-playground/topics/flights")
    
    p = beam.Pipeline(RUNNER, options=opts)
      
    sink = beam.io.BigQuerySink('{}:playground_ds.flight_events'.format(PROJECT), write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND)
    (
      p | 'Read data from Pubsub' >> beam.io.Read(source)
        | 'Get message' >> beam.Map(to_bq_row)
        | 'Write to BigQuery' >> beam.io.Write(sink)
    )

    p.run()

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 16:07:20.717033. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [10]:
run_flights_pipeline()

Launching Dataflow job ingest-flight-events-170829-160721
Check the Dataflow jobs on Google Cloud Console...


ValueError: Pipeline has validations errors: 
Streaming pipelines are not supported.

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 16:07:21.194032. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [44]:

def to_table_row(line):
    import csv
    csv_fields = next(csv.reader([line]))
    
    try:
      data_row = {"airportId":int(csv_fields[0]), "longitude":float(csv_fields[21]), "latitude":float(csv_fields[26])}
    except:
      data_row = {"airportId":-1, "longitude":1.1, "latitude":-1.1}
    return data_row
  

#print(exract_fields("101,2.0334,-856.06"))
  
def run_airports_pipeline():
  
  source = 'git-repo/04_streaming/simulate/airports.csv.gz'
    
  sink = beam.io.BigQuerySink('{}:playground_ds.airport_locations'.format(PROJECT), write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND)
    
  p = beam.Pipeline("DirectRunner")
  
  (
    p | 'Read data from text file' >> beam.io.ReadFromText(source)
      | 'Extract Fields' >> beam.Map(to_table_row)
      | 'Write csv file to GCS' >> beam.io.Write(sink)
  )

  p.run()

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 15:51:20.060037. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [45]:
run_airports_pipeline()

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 15:51:20.453304. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
